### Решение проблемы: Не цепляет CUDA Llama_cpp в Kaggle

Возможно есть варианты проще, но этот вариант помог задействовать GPU с Kaggle

### Проверка драйвера

Начинаем с проверки версии драйвера <br>
На момент написания вижу такое сообщение: <br>Build cuda_12.3.r12.3/compiler.33567101_0<br>Считаю что у меня версия 12.3

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Wed_Nov_22_10:17:15_PST_2023
Cuda compilation tools, release 12.3, V12.3.107
Build cuda_12.3.r12.3/compiler.33567101_0


### Выбор команды установки
На гитхабе данной библиотеки (https://github.com/abetlen/llama-cpp-python) есть описания разных способов установки<br>Нам подходит вариант с выбором **Pre-built Wheel (New)**<br>

----------
**Шаблон команды**: !pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/<cuda-version>
----------
    
Вместо **\<cuda-version\>** нужно подставить один из вариантов ниже:

cu121: CUDA 12.1<br>
cu122: CUDA 12.2<br>
cu123: CUDA 12.3<br>
cu124: CUDA 12.4<br>
    
**В моем случае - 12.3**
    
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123

In [2]:
!pip install llama-cpp-python \
  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu123
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.5/444.5 MB 3.2 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.7 MB/s eta 0:00:00


### Установка доп библиотек - опционально
Далее проверим как работает данный метод и проведу установку дополнительных библиотек langchain

In [3]:
!pip install langchain langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.4 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.2 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-

### Блок импортов для тестирования

In [4]:
from langchain.prompts import PromptTemplate
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain.llms import HuggingFacePipeline
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

### Делаем загрузку готовой модели с HF

In [5]:
import os
from huggingface_hub import hf_hub_download

# Указываю имя репозитория и название скачиваемой модели
model_name = "lmstudio-community/Mistral-Nemo-Instruct-2407-GGUF"
model_file = "Mistral-Nemo-Instruct-2407-Q4_K_M.gguf"


# Загрузка с Hugging Face Hub
model_path = hf_hub_download(
    model_name,
    filename=model_file,
    local_dir='models/',  # Загрузку сделаем в папку "models" - опционально
    token="token"  #тут указываем ваш токен доступа с huggingface (Setting -> Access Toekns -> New token -> Generate Token)
)

print("My model path:", model_path)

Mistral-Nemo-Instruct-2407-Q4_K_M.gguf:   0%|          | 0.00/7.48G [00:00<?, ?B/s]

My model path: models/Mistral-Nemo-Instruct-2407-Q4_K_M.gguf


### Инициализация модели

После инициализации модели вижу, что все две карты подцепились

-----

llm_load_tensors: offloaded 41/41 layers to GPU<br>
llm_load_tensors:        CPU buffer size =   360.00 MiB<br>
llm_load_tensors:      CUDA0 buffer size =  3265.43 MiB<br>
llm_load_tensors:      CUDA1 buffer size =  3497.87 MiB<br>

-----

BLAS = 1<br>

In [6]:
# Путь до модели
model_path = "/kaggle/working/models/Mistral-Nemo-Instruct-2407-Q4_K_M.gguf"

# Инициализирую модель через LlamaCpp
llm = LlamaCpp(
    model_path=model_path,
    temperature=0.75,
    max_tokens=200,
    n_gpu_layers=-1,
)

llama_model_loader: loaded meta data with 39 key-value pairs and 363 tensors from /kaggle/working/models/Mistral-Nemo-Instruct-2407-Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Mistral Nemo Instruct 2407
llama_model_loader: - kv   3:                            general.version str              = 2407
llama_model_loader: - kv   4:                           general.finetune str              = Instruct
llama_model_loader: - kv   5:                           general.basename str              = Mistral-Nemo
llama_model_loader: - kv   6:                         general.size_label str              = 12B
llama_model_loade

### Тестируем модель

Проверяем, что модель действительно быстро отвечает и не делает просчет через CPU

In [7]:

template = """
Ты рускоязычная модель. Отвечаешь только на русском языке

Question: {question}

Answer:"""

prompt = PromptTemplate.from_template(template)
llm_chain = prompt | llm
question = """
На основании текста ниже ответь на вопрос

###
Изобрести гибридный натриево-ионный АКБ удалось исследователям Корейского института науки и технологий (KAIST). Ученые создали элемент питания, который способен заряжаться за несколько секунд, при этом аккумулятор обладает и большой емкостью, и увеличенной плотностью энергии относительно стандартных литий-ионных батарей. Особенность разработки в том, что натрий в тысячи раз имеет большее распространение на Земле, чем литий, поэтому производство АКБ на его основе будет и дешевле, и проще, и перспективнее.
Сообщается, что исследовательская группа объединила материалы анодов с катодами, подходящими для суперконденсаторов. Такой дуэт позволит получившемуся аккумулятору продемонстрировать сверхбыструю зарядку и разрядку с высокой емкостью самой батареи. Опыт же показал, что изобретение жизнеспособно и даже сможет стать альтернативой литий-ионным вариантам. По характеристикам плотности энергии разработка ничуть не уступает суперконденсаторам.
###

О чем говорится в тексте?

"""
# llm_chain.invoke({"question": question}, stop=stop_sequence)
# output = llm.invoke(question)
output = llm_chain.invoke({"question": question})
print(output)


llama_print_timings:        load time =     344.26 ms
llama_print_timings:      sample time =     334.63 ms /   176 runs   (    1.90 ms per token,   525.95 tokens per second)
llama_print_timings: prompt eval time =    4186.01 ms /   335 tokens (   12.50 ms per token,    80.03 tokens per second)
llama_print_timings:        eval time =    6068.79 ms /   175 runs   (   34.68 ms per token,    28.84 tokens per second)
llama_print_timings:       total time =   10821.35 ms /   510 tokens


 В тексте говорится о новом типе аккумуляторной батареи (АКБ), которая была изобретена исследователями Корейского института науки и технологий (KAIST). Эта новая АКБ является гибридным натриево-ионным вариантом, который обладает способностью заряжаться всего за несколько секунд, а также демонстрирует большие значения емкости самой батареи. По сравнению с обычными литий-ионными АКБ, изобретение является более дешевым и простым в производстве, поскольку натрий имеет гораздо большее распространение на Земле, чем литий. По своим характеристикам плотности энергии разработка ничуть не уступает суперконденсаторам.
